In [44]:
import pandas as pd
import numpy as np
import re

In [45]:
pd.set_option("display.max_columns", 85)
pd.set_option("display.max_rows", 85)

### Talkapalooza - TL BL SS 23

# Data preparation:

As a source of audio files for single word recordings, I downloaded the English language dataset from [Lingualibre's Languages Gallery](https://lingualibre.org/wiki/LinguaLibre:About). LinguaLibre is a project by Wikimédia France to collectively and collaboratively preserve and provide oral recordings of languages across the globe.

The English dataset consists of recordings provided by 109 unique speakers, amassing about 33 100 recordings covering 29 000 unique words. About 75% of recordings were provided by male speakers.

To have an overview over the pronunciations provided, as well as to know for which there are several recordings, some data preparation and analysis has to be performed.

## Exploring the dataset:

In [46]:
import os

root = "../.source/lingualibre_en/"

In [58]:
recordings_df = pd.DataFrame(columns=['path', 'name'])
recordings_df.head()

,path,name


In [69]:
for path, subdirs, files in os.walk(root):
    for name in files:
        temp_df = pd.DataFrame([[path, name]], columns=['path', 'name'])
        recordings_df= pd.concat([recordings_df, temp_df])

In [93]:
recordings_df = recordings_df.reset_index()
recordings_df = recordings_df.drop('index', axis=1)

In [102]:
recordings_df.shape

(31756, 2)

In [101]:
recordings_df[::50].head(30)

,path,name
0,../.source/lingualibre_en/Eric.LEWIN,TBD.ogg
50,../.source/lingualibre_en/Fredericknoronha,Rosary College of Commerce and Arts.ogg
100,../.source/lingualibre_en/Opsylac,they.ogg
150,../.source/lingualibre_en/I learned some phrases,chary.ogg
200,../.source/lingualibre_en/I learned some phrases,sear.ogg
250,../.source/lingualibre_en/I learned some phrases,rusts.ogg
300,../.source/lingualibre_en/I learned some phrases,obligatory.ogg
350,../.source/lingualibre_en/I learned some phrases,quintessential.ogg
400,../.source/lingualibre_en/I learned some phrases,philately.ogg
450,../.source/lingualibre_en/I learned some phrases,fresco.ogg
